In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.models import load_model, Model
from keras.callbacks import EarlyStopping
import keras.backend as K
import numpy as np

# our stuff
from synth import *
from utils import *
from const import Tx, Ty, n_a, n_s, input_vocab, output_vocab, output_vocab_inv

import matplotlib.pyplot as plt
%matplotlib inline

/home/opyate/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# check GPU
from tensorflow.python.client import device_lib
[print(x.name) for x in device_lib.list_local_devices()]

/device:CPU:0
/device:GPU:0


[None, None]

In [3]:
# Returns input,output,dt
# Run once to see an example.
load_time()

("twenty five minutes and fifty five seconds past three o'clock in the morning",
 '03:25:55',
 datetime.time(3, 25, 55))

In [4]:
# One-hot representations of load_time()'s input and output
# will be fed into the model.

i, o, _ = load_time()

print('Input:', i)
print('Target:', o)

ioh, ooh = one_hot([i], [o], Tx, Ty, input_vocab, output_vocab)

print('Input shape:', ioh.shape)
print('Target shape:', ooh.shape)
print(*ooh)

Input: 18 hours, 8 mins and 34 seconds
Target: 18:08:34
Input shape: (1, 128, 102)
Target shape: (1, 8, 11)
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


- ioh, "input one-hot", has shape (128,102), i.e. it can accept an input string of up to (or padded to) 128 characters of a possible vocabulary of 102 characters.
- ooh, "output one-hot", has shape (8,11), i.e. the model will predict 8 characters of the form 'hh:mm:ss' of a possible vocabulary of all digits and colon ':'.

In [5]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
# softmax(axis = 1) from utils.py
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [6]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e.
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas"
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas, a])
    
    return context

In [7]:
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(output_vocab), activation=softmax)

In [8]:
def get_model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    # Step 1: Define pre-attention Bi-LSTM. Remember to use return_sequences=True.
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    return model

In [9]:
model = get_model(Tx, Ty, n_a, n_s, len(input_vocab), len(output_vocab))

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 102)     0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 128, 128)     85504       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 128, 128)     0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

In [11]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=2, verbose=0, mode='auto')
data_maker = dataset_generator(Tx, Ty, input_vocab, output_vocab, n_s, batch_size=1000)
model.fit_generator(data_maker, steps_per_epoch=1000, epochs=1000, callbacks=[early_stop])

Epoch 1/1000
1000/1000 [==============================] - 251s 251ms/step - loss: 6.7304 - dense_2_loss_1: 0.6983 - dense_2_loss_2: 1.7767 - dense_2_loss_3: 0.0563 - dense_2_loss_4: 0.9313 - dense_2_loss_5: 1.1720 - dense_2_loss_6: 0.0831 - dense_2_loss_7: 0.9671 - dense_2_loss_8: 1.0457 - dense_2_acc_1: 0.6624 - dense_2_acc_2: 0.3198 - dense_2_acc_3: 0.9990 - dense_2_acc_4: 0.6168 - dense_2_acc_5: 0.5652 - dense_2_acc_6: 0.9990 - dense_2_acc_7: 0.5942 - dense_2_acc_8: 0.6026
Epoch 2/1000
1000/1000 [==============================] - 248s 248ms/step - loss: 1.1448 - dense_2_loss_1: 0.2080 - dense_2_loss_2: 0.5064 - dense_2_loss_3: 9.8783e-04 - dense_2_loss_4: 0.1322 - dense_2_loss_5: 0.1285 - dense_2_loss_6: 9.7126e-04 - dense_2_loss_7: 0.1063 - dense_2_loss_8: 0.0614 - dense_2_acc_1: 0.9203 - dense_2_acc_2: 0.8650 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 0.9595 - dense_2_acc_5: 0.9685 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 0.9758 - dense_2_acc_8: 0.9891
Epoch 3/1000
1000/1000 [======

1000/1000 [==============================] - 247s 247ms/step - loss: 0.0127 - dense_2_loss_1: 2.6969e-04 - dense_2_loss_2: 5.9767e-04 - dense_2_loss_3: 8.6841e-07 - dense_2_loss_4: 1.5116e-04 - dense_2_loss_5: 0.0115 - dense_2_loss_6: 1.1560e-06 - dense_2_loss_7: 1.0697e-04 - dense_2_loss_8: 7.0767e-05 - dense_2_acc_1: 1.0000 - dense_2_acc_2: 1.0000 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 0.9920 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000
Epoch 34/1000
1000/1000 [==============================] - 247s 247ms/step - loss: 0.0131 - dense_2_loss_1: 3.3803e-04 - dense_2_loss_2: 8.1042e-04 - dense_2_loss_3: 7.2164e-07 - dense_2_loss_4: 1.4422e-04 - dense_2_loss_5: 0.0116 - dense_2_loss_6: 1.1019e-06 - dense_2_loss_7: 1.1393e-04 - dense_2_loss_8: 7.5664e-05 - dense_2_acc_1: 0.9999 - dense_2_acc_2: 0.9999 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 1.0000 - dense_2_acc_5: 0.9918 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1

In [13]:
EXAMPLES = [
    'the time now is noon',
    "it is five o'clock",
    "half past eleven o'clock in the morning",
    "twenty nine minutes and fifty nine seconds past noon",
    "twenty nine minutes and fifty nine seconds past eleven o'clock at night",
    "it is twenty eight minutes to noon"
]

np.set_printoptions(threshold=10)
#print(human_vocab)
#print('human', len(human_vocab))
for example in EXAMPLES:
    print()
    print(example)
    source = string_to_int(example, Tx, input_vocab)
    
    #print(source)
    
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(input_vocab)), source))) #.swapaxes(0,1)
             
    #print(source.shape)
    #print(source)
    
    prediction = model.predict([np.expand_dims(source, axis=0), np.zeros((1,n_s)), np.zeros((1,n_s))])
    prediction = np.argmax(prediction, axis = -1)
    output = [output_vocab_inv[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))
    


the time now is noon
source: the time now is noon
output: 11:58:00

it is five o'clock
source: it is five o'clock
output: 13:09:55

half past eleven o'clock in the morning
source: half past eleven o'clock in the morning
output: 11:30:00

twenty nine minutes and fifty nine seconds past noon
source: twenty nine minutes and fifty nine seconds past noon
output: 12:29:59

twenty nine minutes and fifty nine seconds past eleven o'clock at night
source: twenty nine minutes and fifty nine seconds past eleven o'clock at night
output: 23:29:59

it is twenty eight minutes to noon
source: it is twenty eight minutes to noon
output: 13:59:22


In [26]:
# happy?

import os

modelfile = "themodel.h5"
if os.path.isfile(modelfile):
    print('already exists', modelfile)
else:
    print('saving', modelfile)
    model.save(modelfile)

modelweightsfile = "themodel-weights.h5"
if os.path.isfile(modelweightsfile):
    print('already exists', modelweightsfile)
else:
    print('saving', modelweightsfile)
    model.save_weights(modelweightsfile)


already exists themodel.h5
already exists themodel-weights.h5


See `attention.ipynb` for the attention map.